In [566]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import re
from catboost import CatBoostRegressor
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import StackingRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import Ridge

In [568]:
df = pd.read_csv('Cars Datasets 2025.csv', encoding='ISO-8859-1') 

# ℹ️ | **Information**

In [569]:
df.head()

,Company Names,Cars Names,Engines,CC/Battery Capacity,HorsePower,Total Speed,Performance(0 - 100 )KM/H,Cars Prices,Fuel Types,Seats,Torque
0,FERRARI,SF90 STRADALE,V8,3990 cc,963 hp,340 km/h,2.5 sec,"$1,100,000",plug in hyrbrid,2,800 Nm
1,ROLLS ROYCE,PHANTOM,V12,6749 cc,563 hp,250 km/h,5.3 sec,"$460,000",Petrol,5,900 Nm
2,Ford,KA+,1.2L Petrol,"1,200 cc",70-85 hp,165 km/h,10.5 sec,"$12,000-$15,000",Petrol,5,100 - 140 Nm
3,MERCEDES,GT 63 S,V8,"3,982 cc",630 hp,250 km/h,3.2 sec,"$161,000",Petrol,4,900 Nm
4,AUDI,AUDI R8 Gt,V10,"5,204 cc",602 hp,320 km/h,3.6 sec,"$253,290",Petrol,2,560 Nm


In [570]:
df[df['Company Names'] == 'Jaguar Land Rover']

,Company Names,Cars Names,Engines,CC/Battery Capacity,HorsePower,Total Speed,Performance(0 - 100 )KM/H,Cars Prices,Fuel Types,Seats,Torque
899,Jaguar Land Rover,Jaguar F-Type,V6 / V8 Petrol,2995 cc (V6),335 hp,260 km/h,5.1 sec,"$70,000",Petrol,2,450 Nm
900,Jaguar Land Rover,Jaguar XE,I4 Petrol,1999 cc (I4),250 hp,250 km/h,6.5 sec,"$45,000",Petrol,5,365 Nm
901,Jaguar Land Rover,Jaguar XF,I4 Petrol,1999 cc (I4),250 hp,250 km/h,6.5 sec,"$50,000",Petrol,5,365 Nm
902,Jaguar Land Rover,Jaguar XJ,V6 Petrol,2995 cc (V6),300 hp,250 km/h,6.0 sec,"$85,000",Petrol,5,400 Nm
903,Jaguar Land Rover,Jaguar E-PACE,I4 Petrol,1999 cc (I4),200 hp,230 km/h,7.0 sec,"$50,000",Petrol,5,430 Nm
904,Jaguar Land Rover,Jaguar I-PACE,Electric (Dual Motor),90 kWh (Battery),400 hp,200 km/h,4.8 sec,"$70,000",Electric,5,696 Nm
905,Jaguar Land Rover,Jaguar F-PACE,I4 Petrol / V6 Petrol,1999 cc (I4) / 2995 cc (V6),250 hp / 340 hp,230 km/h,7.2 sec / 5.4 sec,"$55,000 / $65,000",Petrol,5,365 Nm / 480 Nm
906,Jaguar Land Rover,Jaguar X-TYPE,I4 Petrol,1998 cc (I4),160 hp,215 km/h,9.0 sec,"$40,000",Petrol,5,215 Nm
907,Jaguar Land Rover,Jaguar F-PACE R-Dynamic,I4 Petrol / V6 Petrol,1999 cc (I4) / 2995 cc (V6),340 hp,250 km/h,5.4 sec,"$65,000",Petrol,5,480 Nm
908,Jaguar Land Rover,Jaguar XE R-Dynamic,I4 Petrol,1999 cc (I4),340 hp,250 km/h,5.4 sec,"$60,000",Petrol,5,450 Nm


In [571]:
df.shape

(1218, 11)

In [572]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1218 entries, 0 to 1217
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Company Names              1218 non-null   object
 1   Cars Names                 1218 non-null   object
 2   Engines                    1218 non-null   object
 3   CC/Battery Capacity        1215 non-null   object
 4   HorsePower                 1218 non-null   object
 5   Total Speed                1218 non-null   object
 6   Performance(0 - 100 )KM/H  1212 non-null   object
 7   Cars Prices                1218 non-null   object
 8   Fuel Types                 1218 non-null   object
 9   Seats                      1218 non-null   object
 10  Torque                     1217 non-null   object
dtypes: object(11)
memory usage: 104.8+ KB


In [573]:
df.describe()

,Company Names,Cars Names,Engines,CC/Battery Capacity,HorsePower,Total Speed,Performance(0 - 100 )KM/H,Cars Prices,Fuel Types,Seats,Torque
count,1218,1218,1218,1215,1218,1218,1212,1218,1218,1218,1217
unique,37,1201,356,311,456,114,180,535,23,19,263
top,Nissan,Macan T,I4,"2,000 cc",355 hp,250 km/h,6.5 sec,"$35,000",Petrol,5,400 Nm
freq,149,2,64,31,23,145,45,36,871,692,72


In [574]:
df.isnull().sum()

Company Names                0
Cars Names                   0
Engines                      0
CC/Battery Capacity          3
HorsePower                   0
Total Speed                  0
Performance(0 - 100 )KM/H    6
Cars Prices                  0
Fuel Types                   0
Seats                        0
Torque                       1
dtype: int64

In [575]:
df.dtypes

Company Names                object
Cars Names                   object
Engines                      object
CC/Battery Capacity          object
HorsePower                   object
Total Speed                  object
Performance(0 - 100 )KM/H    object
Cars Prices                  object
Fuel Types                   object
Seats                        object
Torque                       object
dtype: object

In [576]:
df.fillna(0, inplace=True)

In [577]:
df.duplicated().sum()

4

In [578]:
df[df.duplicated()]

,Company Names,Cars Names,Engines,CC/Battery Capacity,HorsePower,Total Speed,Performance(0 - 100 )KM/H,Cars Prices,Fuel Types,Seats,Torque
336,Volkswagen,Golf Cabriolet,1.2L I4 Turbo / 2.0L I4 Turbo,1197 cc / 1984 cc,105 - 210 hp,217 km/h,8.4 sec,"$30,000",Petrol,4,200 Nm
354,Volkswagen,Jetta Hybrid,1.4L I4 Turbo + Electric Motor,1395 cc,170 hp,210 km/h,8.6 sec,"$28,000",Hybrid,5,250 Nm
658,Tata Motors,Tiago EV,Permanent Magnet Synchronous,24 kWh,74 hp,120 km/h,5.7 sec,"$11,000",Electric,5,114 Nm
755,Chevrolet,Tahoe RST,5.3L V8 Gasoline,5300 cc,355 hp,180 km/h,7.4 sec,"$64,100",Petrol,7,519 Nm


# 🧹 | **Preprocessing**

In [579]:
df['Company Names'] = df['Company Names'].str.lower()
df['Cars Names'] = df['Cars Names'].str.lower()
df['Performance(0 - 100 )KM/H'] = df['Performance(0 - 100 )KM/H'].str.replace(' sec', '')
df['HorsePower'] = df['HorsePower'].str.replace(' hp', '')
df['Total Speed'] = df['Total Speed'].str.replace(' km/h', '')
df['Torque'] = df['Torque'].str.replace(' Nm', '')
df['Cars Prices'] = df['Cars Prices'].str.replace('$', '').str.replace(',', '')
df

,Company Names,Cars Names,Engines,CC/Battery Capacity,HorsePower,Total Speed,Performance(0 - 100 )KM/H,Cars Prices,Fuel Types,Seats,Torque
0,ferrari,sf90 stradale,V8,3990 cc,963,340,2.5,1100000,plug in hyrbrid,2,800
1,rolls royce,phantom,V12,6749 cc,563,250,5.3,460000,Petrol,5,900
2,ford,ka+,1.2L Petrol,"1,200 cc",70-85,165,10.5,12000-15000,Petrol,5,100 - 140
3,mercedes,gt 63 s,V8,"3,982 cc",630,250,3.2,161000,Petrol,4,900
4,audi,audi r8 gt,V10,"5,204 cc",602,320,3.6,253290,Petrol,2,560
...,...,...,...,...,...,...,...,...,...,...,...
1213,toyota,crown signia,2.5L Hybrid I4,2487 cc,240,180,7.6,43590  48000,Hybrid (Gas + Electric),5,239
1214,toyota,4runner (6th gen),2.4L Turbo I4 (i-FORCE MAX Hybrid),2393 cc + Battery,326,180,6.8,50000,Hybrid,7,630
1215,toyota,corolla cross,2.0L Gas / 2.0L Hybrid,1987 cc / Hybrid batt,169  196,190,8.0  9.2,25210  29135,Gas / Hybrid,5,190  210
1216,toyota,c-hr+,1.8L / 2.0L Hybrid,1798 / 1987 cc + batt,140  198,180,7.9  10.5, 33000,Hybrid,5,190  205


In [580]:
def clean_numeric_cell(cell):
    if pd.isnull(cell):
        return np.nan
    cell = str(cell).replace(",", "")
    numbers = re.findall(r'\d+(?:\.\d+)?', cell)
    numbers = [float(num) for num in numbers]
    return sum(numbers) / len(numbers) if numbers else np.nan

# List of columns to clean
columns_to_clean = [
    'Cars Prices'
]

# Apply the function to each column
for col in columns_to_clean:
    df[col] = df[col].apply(clean_numeric_cell)


In [581]:
def clean_dataset(df):    
    def parse_intervals(col):
        
        try:
            col = str(col)
            cleaned = re.sub(r'[^\d.,-]', '', col)
            cleaned = cleaned.replace(',', '')
            
            if '-' in cleaned or '–' in cleaned:
                cleaned = cleaned.replace('–', '-')
                cleaned = cleaned.replace('', '-')
                parts = cleaned.split('-')
                parts = [p.strip() for p in parts if p.strip()]
                
                if len(parts) == 2:
                    low = float(parts[0])
                    high = float(parts[1])
                    return (low + high) / 2
                return float(parts[0]) if parts else None
            return float(cleaned) if cleaned else None
        except:
            return None

    numeric_cols = {
        'Cars Prices': None,
        'Torque': None,
        'HorsePower': None,
        'Performance(0 - 100 )KM/H': None
    }
    
    for col in numeric_cols:
        if col in df.columns:
            df[col] = df[col].apply(parse_intervals)

    units_to_remove = {
        'Performance(0 - 100 )KM/H': ' sec',
        'HorsePower': ' hp',
        'Total Speed': ' km/h',
        'Torque': ' Nm'
    }
    
    for col, unit in units_to_remove.items():
        if col in df.columns:
            df[col] = df[col].astype(str).str.replace(unit, '', regex=False)
            try:
                df[col] = pd.to_numeric(df[col])
            except:
                pass
    
    return df

df = clean_dataset(df)
df.tail(10)

,Company Names,Cars Names,Engines,CC/Battery Capacity,HorsePower,Total Speed,Performance(0 - 100 )KM/H,Cars Prices,Fuel Types,Seats,Torque
1208,mazda,bongo truck,2.0L Inline-4,"1,998 cc",120.0,130,15.0,16500.0,Petrol,2,150.0
1209,mazda,rotary pickup,1.3L Wankel Rotary Engine,"1,308 cc",130.0,150,14.5,20000.0,Petrol,2,160.0
1210,mazda,luce rotary coupe,1.3L Wankel Rotary Engine,"1,308 cc",135.0,175,12.0,27500.0,Petrol,4,180.0
1211,mazda,millenia s,2.5L V6,"2,500 cc",200.0,220,8.0,32500.0,Petrol,5,250.0
1212,mazda,787b (race car),2.6L Wankel Rotary Engine,"2,620 cc",700.0,320,3.5,6000000.0,Petrol,1,500.0
1213,toyota,crown signia,2.5L Hybrid I4,2487 cc,240.0,180,7.6,45795.0,Hybrid (Gas + Electric),5,239.0
1214,toyota,4runner (6th gen),2.4L Turbo I4 (i-FORCE MAX Hybrid),2393 cc + Battery,326.0,180,6.8,50000.0,Hybrid,7,630.0
1215,toyota,corolla cross,2.0L Gas / 2.0L Hybrid,1987 cc / Hybrid batt,169196.0,190,nan,27172.5,Gas / Hybrid,5,190210.0
1216,toyota,c-hr+,1.8L / 2.0L Hybrid,1798 / 1987 cc + batt,140198.0,180,nan,33000.0,Hybrid,5,190205.0
1217,toyota,rav4 (6th gen),2.5L Hybrid / Plug-in Hybrid,2487 cc + batt,219302.0,200,nan,36000.0,Hybrid / Plug-in,5,221400.0


In [582]:
df = df.drop_duplicates()

In [583]:
df['Fuel Types'].unique()

array(['plug in hyrbrid', 'Petrol', 'Diesel', 'Hybrid', 'Electric',
       'Petrol/Diesel', 'Plug-in Hybrid', 'Petrol/AWD', 'Petrol/Hybrid',
       'Hydrogen', 'Diesel/Petrol', 'Petrol/EV', 'Hybrid/Electric',
       'Petrol, Hybrid', 'Petrol, Diesel', 'Hybrid (Petrol)',
       'CNG/Petrol', 'Hybrid/Petrol', 'Diesel Hybrid', 'Petrol (Hybrid)',
       'Hybrid (Gas + Electric)', 'Gas / Hybrid', 'Hybrid / Plug-in'],
      dtype=object)

Are the same:
* 'Petrol/Hybrid', 'Petrol, Hybrid', 'Hybrid/Petrol', 'Petrol (Hybrid)' 
* 'Hybrid (Gas + Electric)', 'Gas / Hybrid'

In [584]:
def normalize_fuel_type(item):
    item = str(item).lower().strip()
    item = re.sub(r'[^\w\s-]', ' ', item) 
    item = re.sub(r'plug[-\s]?in', 'plugin', item)
    
    replacements = {
        'gas': 'petrol',
        'ev': 'electric',
        'hyrbrid': 'hybrid',
        'cng': 'compressed_natural_gas'
    }
    for wrong, correct in replacements.items():
        item = item.replace(wrong, correct)

    separators_re = re.compile(r'[,/&+]|\band\b|\s+')
    parts = [p.strip() for p in separators_re.split(item) if p.strip()]

    final_parts = []
    i = 0
    while i < len(parts):
        if parts[i] == 'plugin' and i+1 < len(parts) and parts[i+1] == 'hybrid':
            final_parts.append('plugin hybrid')
            i += 2
        else:
            final_parts.append(parts[i])
            i += 1
    
    return final_parts

mlb = MultiLabelBinarizer()
fuel_encoded = mlb.fit_transform(df['Fuel Types'].apply(normalize_fuel_type))
df = df.drop(columns=['Fuel Types'])
df[mlb.classes_] = fuel_encoded
df.tail(10)

,Company Names,Cars Names,Engines,CC/Battery Capacity,HorsePower,Total Speed,Performance(0 - 100 )KM/H,Cars Prices,Seats,Torque,awd,compressed_natural_gas,diesel,electric,hybrid,hydrogen,petrol,plugin,plugin hybrid
1208,mazda,bongo truck,2.0L Inline-4,"1,998 cc",120.0,130,15.0,16500.0,2,150.0,0,0,0,0,0,0,1,0,0
1209,mazda,rotary pickup,1.3L Wankel Rotary Engine,"1,308 cc",130.0,150,14.5,20000.0,2,160.0,0,0,0,0,0,0,1,0,0
1210,mazda,luce rotary coupe,1.3L Wankel Rotary Engine,"1,308 cc",135.0,175,12.0,27500.0,4,180.0,0,0,0,0,0,0,1,0,0
1211,mazda,millenia s,2.5L V6,"2,500 cc",200.0,220,8.0,32500.0,5,250.0,0,0,0,0,0,0,1,0,0
1212,mazda,787b (race car),2.6L Wankel Rotary Engine,"2,620 cc",700.0,320,3.5,6000000.0,1,500.0,0,0,0,0,0,0,1,0,0
1213,toyota,crown signia,2.5L Hybrid I4,2487 cc,240.0,180,7.6,45795.0,5,239.0,0,0,0,1,1,0,1,0,0
1214,toyota,4runner (6th gen),2.4L Turbo I4 (i-FORCE MAX Hybrid),2393 cc + Battery,326.0,180,6.8,50000.0,7,630.0,0,0,0,0,1,0,0,0,0
1215,toyota,corolla cross,2.0L Gas / 2.0L Hybrid,1987 cc / Hybrid batt,169196.0,190,nan,27172.5,5,190210.0,0,0,0,0,1,0,1,0,0
1216,toyota,c-hr+,1.8L / 2.0L Hybrid,1798 / 1987 cc + batt,140198.0,180,nan,33000.0,5,190205.0,0,0,0,0,1,0,0,0,0
1217,toyota,rav4 (6th gen),2.5L Hybrid / Plug-in Hybrid,2487 cc + batt,219302.0,200,nan,36000.0,5,221400.0,0,0,0,0,1,0,0,1,0


In [585]:
df['Seats'].unique()

array(['2', '5', '4', '2+2', '7', '8', '20', '15', '12', '3', '9', '6',
       '1', '2\x9615', '2\x967', '2\x966', '7\x968', '2\x969', '2\x9612'],
      dtype=object)

In [586]:
def clean_seats(item):
    replacements = {
        '2\x9615': '2',
        '2\x967': '2',
        '2\x966': '2',
        '7\x968': '2',
        '2\x969': '2',
        '2\x9612': '2',
        '2+2': '4'
    }

    for wrong, correct in replacements.items():
        item = item.replace(wrong, correct)
    
    cleaned = ''.join(c for c in item if c.isdigit() or c == '+')
    
    if '+' in cleaned:
        parts = cleaned.split('+')
        try:
            return sum(int(p) for p in parts)
        except:
            return 0
    
    try:
        return int(cleaned) if cleaned else 0
    except:
        return 0

df['Seats'] = df['Seats'].apply(clean_seats)

In [587]:
df['CC/Battery Capacity'].unique()

array(['3990 cc', '6749 cc', '1,200 cc', '3,982 cc', '5,204 cc',
       '3,994 cc', '3,996 cc', '6,498 cc', '3,900 cc', '6496 cc',
       '6,496 cc', '2,998 cc', '1,998 cc', '2,387 cc', '5,663 cc',
       '3,799 cc', '3,696 cc', '2,488 cc', '3,498 cc', '5,598 cc',
       '5,935 cc', '3,998 cc', '6,592 cc', '6,749 cc', '1991 cc',
       '2,996 cc', '1,991 cc', 0, '5,980 cc', '4,395 cc', '2,993 cc',
       '4,935 cc', '3,993 cc', '2,995 cc', '1,984 cc', '95 kwh',
       '11.6 kwh', '1,499 cc', '1,496 cc', '1,995 cc', '1,598 cc',
       '1,969 cc', '2,480 cc', '1,996 cc', '1,600 cc / 13.8 kwh',
       '1,498 cc', '1,600 cc / 1.49 kwh', '1,598 cc / 13.8 kwh',
       '3,471 cc', '3,500 cc', '1,798 cc', '2,494 cc', '3,456 cc',
       '3,956 cc', '2,694 cc', '1,986 cc',
       '64.8 kwh (Base Model),72.8 kwh (Higher End Model', '1,497 cc',
       '1,197 cc', '1,396 cc', '1,999 cc', '2,179 cc', '1,997 cc',
       '1,462 cc', '1,618 cc', '3,346 cc', '3,445 cc', '1,987 cc',
       '1.24 kwh', '2

In [588]:
def convert_power(value):
    """
    Улучшенная обработка всех форматов:
    - Диапазоны (58-77 cc, 40-58 kWh)
    - Одиночные значения
    - Смешанные случаи (cc + kWh)
    - Учет battery как признака гибрида
    """
    if pd.isna(value) or str(value).lower() == 'nan':
        return pd.Series({'Engine_CC': np.nan, 'Battery_W': np.nan})
    
    value = str(value).lower()
    cc_values = []
    kwh_values = []
    has_battery = 'battery' in value or 'batt' in value
    
    # 1. Обработка диапазонов cc (58-77 cc)
    range_cc = re.findall(r'(\d+)\s*-\s*(\d+)\s*cc', value)
    for start, end in range_cc:
        cc_values.extend([float(start), float(end)])
    
    # 2. Одиночные значения cc (улучшенное выражение)
    single_cc = re.findall(r'(?<![\d-])(\d+[,.]?\d*)\s*cc(?!\s*kwh)', value.replace(',', ''))
    cc_values.extend([float(x) for x in single_cc])
    
    # 3. Обработка диапазонов kwh (40-58 kwh)
    range_kwh = re.findall(r'(\d+\.?\d*)\s*-\s*(\d+\.?\d*)\s*kwh', value)
    for start, end in range_kwh:
        kwh_values.extend([float(start), float(end)])
    
    # 4. Одиночные значения kwh
    single_kwh = re.findall(r'(?<![\d-])(\d+[,.]?\d*)\s*kwh', value.replace(',', ''))
    kwh_values.extend([float(x) for x in single_kwh])
    
    # 5. Если есть battery, но нет kwh
    if has_battery and not kwh_values:
        batt_num = re.search(r'(\d+[,.]?\d*)\s*(?:kwh\s*)?batt', value)
        if batt_num:
            kwh_values.append(float(batt_num.group(1)))
    
    # Расчет средних значений
    engine_cc = round(np.mean(cc_values)) if cc_values else np.nan
    battery_w = round(np.mean(kwh_values)*1000) if kwh_values else np.nan
    
    return pd.Series({
        'Engine_CC': engine_cc,
        'Battery_W': battery_w
    })

df[['Engine_CC', 'Battery_W']] = df['CC/Battery Capacity'].apply(convert_power)


In [589]:
df['Battery_W'].describe()

count       110.000000
mean      77631.427273
std       48801.550128
min        1240.000000
25%       50000.000000
50%       77400.000000
75%       93400.000000
max      212000.000000
Name: Battery_W, dtype: float64

In [590]:
def fill_hybrid_battery(df, battery_col='Battery_W', source_col='CC/Battery Capacity'):

    # Копируем датафрейм для безопасного изменения
    df_filled = df.copy()
    
    # 1. Определяем гибриды (есть cc И указание на батарею/гибрид)
    hybrid_mask = (
        df_filled[source_col].str.contains(r'\d+\s*cc', case=False, na=False) & 
        df_filled[source_col].str.contains('battery|batt|hybrid', case=False, na=False)
    )
    
    # 2. Вычисляем среднюю мощность известных гибридов
    known_hybrids = df_filled[hybrid_mask & df_filled[battery_col].notna()]
    mean_battery = known_hybrids[battery_col].mean()
    
    # 3. Если нет данных, используем типичное значение для гибридов (например, 15 kWh)
    if pd.isna(mean_battery):
        mean_battery = 15000  # 15 kWh в ваттах
    
    # 4. Заполняем пропуски у гибридов
    fill_mask = hybrid_mask & df_filled[battery_col].isna()
    df_filled.loc[fill_mask, battery_col] = mean_battery
    
    return df_filled
df = fill_hybrid_battery(df)

In [591]:
df=df.drop('CC/Battery Capacity', axis=1)
df

,Company Names,Cars Names,Engines,HorsePower,Total Speed,Performance(0 - 100 )KM/H,Cars Prices,Seats,Torque,awd,compressed_natural_gas,diesel,electric,hybrid,hydrogen,petrol,plugin,plugin hybrid,Engine_CC,Battery_W
0,ferrari,sf90 stradale,V8,963.0,340,2.5,1100000.0,2,800.0,0,0,0,0,0,0,0,0,1,3990.0,NaN
1,rolls royce,phantom,V12,563.0,250,5.3,460000.0,5,900.0,0,0,0,0,0,0,1,0,0,6749.0,NaN
2,ford,ka+,1.2L Petrol,77.5,165,10.5,13500.0,5,120.0,0,0,0,0,0,0,1,0,0,1200.0,NaN
3,mercedes,gt 63 s,V8,630.0,250,3.2,161000.0,4,900.0,0,0,0,0,0,0,1,0,0,3982.0,NaN
4,audi,audi r8 gt,V10,602.0,320,3.6,253290.0,2,560.0,0,0,0,0,0,0,1,0,0,5204.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1213,toyota,crown signia,2.5L Hybrid I4,240.0,180,7.6,45795.0,5,239.0,0,0,0,1,1,0,1,0,0,2487.0,NaN
1214,toyota,4runner (6th gen),2.4L Turbo I4 (i-FORCE MAX Hybrid),326.0,180,6.8,50000.0,7,630.0,0,0,0,0,1,0,0,0,0,2393.0,20000.0
1215,toyota,corolla cross,2.0L Gas / 2.0L Hybrid,169196.0,190,nan,27172.5,5,190210.0,0,0,0,0,1,0,1,0,0,1987.0,20000.0
1216,toyota,c-hr+,1.8L / 2.0L Hybrid,140198.0,180,nan,33000.0,5,190205.0,0,0,0,0,1,0,0,0,0,1987.0,20000.0


In [592]:
import pandas as pd
import numpy as np

def clean_power_column(power_series):
    cleaned = power_series.copy()
    cleaned = cleaned.str.replace(r'\(est\.\)', '', regex=True)
    cleaned = cleaned.str.extract(r'(\d+)')[0]
    cleaned = pd.to_numeric(cleaned, errors='coerce')
    cleaned = cleaned.astype(pd.Int64Dtype()) 
    return cleaned

df['Total Speed'] = clean_power_column(df['Total Speed'])
df['Performance(0 - 100 )KM/H'] = clean_power_column(df['Performance(0 - 100 )KM/H'])
df['Torque'] = clean_power_column(df['Torque'])

In [593]:
df.dtypes

Company Names                 object
Cars Names                    object
Engines                       object
HorsePower                   float64
Total Speed                    Int64
Performance(0 - 100 )KM/H      Int64
Cars Prices                  float64
Seats                          int64
Torque                         Int64
awd                            int32
compressed_natural_gas         int32
diesel                         int32
electric                       int32
hybrid                         int32
hydrogen                       int32
petrol                         int32
plugin                         int32
plugin hybrid                  int32
Engine_CC                    float64
Battery_W                    float64
dtype: object

In [594]:
df['Engines'].nunique()

356

In [595]:
def clean_engine_type(engine_str):
    engine_str = str(engine_str).strip().lower()
    engine_str = re.sub(r'\d+\.\d+l', '', engine_str)
    engine_str = re.sub(r'[^a-z0-9+]', ' ', engine_str)
    
    replacements = {
        'v8': 'V8',
        'v6': 'V6',
        'v10': 'V10',
        'v12': 'V12',
        'i3': 'I3',
        'i4': 'I4',
        'i6': 'I6',
        'boxer4': 'Boxer-4',
        'electric': 'Electric',
        'hybrid': 'Hybrid',
        'turbo': 'Turbo',
        'diesel': 'Diesel',
        'petrol': 'Petrol',
        'hydrogen fuel cell': 'Hydrogen',
        'plug in hybrid': 'PHEV',
        'wankel': 'Rotary',
        'rotary': 'Rotary'
    }
    
    for key, value in replacements.items():
        if key in engine_str:
            return value
        
    return engine_str.title() if engine_str else "Other"

df['Engines'] = df['Engines'].apply(clean_engine_type)

In [596]:
df['Engines'].nunique()

37

In [597]:
print(f"Пропусков в ценах: {df['Cars Prices'].isna().sum()}")

df = df.dropna(subset=['Cars Prices'])

Пропусков в ценах: 1


In [ ]:
numeric_cols = ['HorsePower', 'Total Speed', 'Performance(0 - 100 )KM/H', 
               'Seats', 'Torque', 'Engine_CC', 'Battery_W']
for col in numeric_cols:
    df[col] = df[col].fillna(df[col].median())

categorical_cols = ['Company Names', 'Cars Names', 'Engines']
for col in categorical_cols:
    df[col] = df[col].fillna(df[col].mode()[0])

C:\Users\Asus\AppData\Local\Temp\ipykernel_25084\3778044951.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Asus\AppData\Local\Temp\ipykernel_25084\3778044951.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Asus\AppData\Local\Temp\ipykernel_25084\3778044951.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-d

In [ ]:
df['power_to_cc'] = df['HorsePower'] / df['Engine_CC']
df['is_luxury'] = (df['Cars Prices'] > df['Cars Prices'].quantile(0.9)).astype(int)

C:\Users\Asus\AppData\Local\Temp\ipykernel_25084\3979503799.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Asus\AppData\Local\Temp\ipykernel_25084\3979503799.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
price_99 = df['Cars Prices'].quantile(0.99)
df = df[df['Cars Prices'] <= price_99]
df['log_price'] = np.log1p(df['Cars Prices'])
y = df['log_price']  

In [ ]:
from category_encoders import TargetEncoder

encoder = TargetEncoder()
df[categorical_cols] = encoder.fit_transform(df[categorical_cols], df['Cars Prices'])

# 📊 | **EDA**

In [602]:
company_counts = df['Company Names'].value_counts().reset_index()
company_counts.columns = ['Company Names', 'count'] 

fig = px.bar(
    company_counts,
    x='Company Names',
    y='count', 
    title='Number of cars by company',
    labels={'Company Names': 'company name', 'count': 'count'},
    color='count',
    color_continuous_scale='Viridis'
)

fig.update_layout(
    xaxis_tickangle=-45,
    hovermode='x unified',
    plot_bgcolor='white',
)

fig.update_layout(
    plot_bgcolor="#23233A",  
    paper_bgcolor="#0f1022", 
    font=dict(color='white'))
fig.show()

In [603]:
company_counts = df['Seats'].value_counts().reset_index()
company_counts.columns = ['Seats', 'count'] 
company_counts = company_counts.sort_values('count', ascending=False)

fig = px.bar(
    company_counts,
    x='Seats',
    y='count', 
    title='Number of cars by seats',
    labels={'Seats': 'seats', 'count': 'count'},
    color='count',
    color_continuous_scale='plotly3'
)
fig.update_layout(
    plot_bgcolor="#252640",  
    paper_bgcolor="#101021", 
    font=dict(color='white'))
fig.show()

In [604]:
fig = px.box(df['Cars Prices'], y="Cars Prices")
fig.update_layout(
    plot_bgcolor="#252640",  
    paper_bgcolor="#101021", 
    font=dict(color='white'))
fig.show()

# 🤖 | **Model**

In [605]:
df

,Company Names,Cars Names,Engines,HorsePower,Total Speed,Performance(0 - 100 )KM/H,Cars Prices,Seats,Torque,awd,...,hybrid,hydrogen,petrol,plugin,plugin hybrid,Engine_CC,Battery_W,power_to_cc,is_luxury,log_price
0,187645.513214,212365.568620,119185.656027,963.0,340,2,1100000.0,2,800,0,...,0,0,0,0,1,3990.0,77000.0,0.241353,1,13.910822
1,301995.768056,129096.145027,486508.713740,563.0,250,5,460000.0,5,900,0,...,0,0,1,0,0,6749.0,77000.0,0.083420,1,13.038984
2,55114.817045,71002.711224,61618.308670,77.5,165,10,13500.0,5,120,0,...,0,0,1,0,0,1200.0,77000.0,0.064583,0,9.510519
3,82011.477893,90193.711193,119185.656027,630.0,250,3,161000.0,4,900,0,...,0,0,1,0,0,3982.0,77000.0,0.158212,0,11.989166
4,81318.755355,102201.422292,137115.257918,602.0,320,3,253290.0,2,560,0,...,0,0,1,0,0,5204.0,77000.0,0.115680,1,12.442294
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1213,44181.593536,75204.564404,36922.839988,240.0,180,7,45795.0,5,239,0,...,1,0,1,0,0,2487.0,77000.0,0.096502,0,10.731952
1214,44181.593536,75751.670539,36922.839988,326.0,180,6,50000.0,7,630,0,...,1,0,0,0,0,2393.0,20000.0,0.136231,0,10.819798
1215,44181.593536,72781.619340,48842.949553,169196.0,190,7,27172.5,5,190210,0,...,1,0,1,0,0,1987.0,20000.0,85.151485,0,10.209998
1216,44181.593536,73539.826474,48842.949553,140198.0,180,7,33000.0,5,190205,0,...,1,0,0,0,0,1987.0,20000.0,70.557625,0,10.404293


In [ ]:
X = df.drop('Cars Prices', axis=1)
y = df['Cars Prices']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Initialize CatBoost model
catboost_model = CatBoostRegressor(
    iterations=2000,
    learning_rate=0.03,
    depth=8,
    l2_leaf_reg=5,
    loss_function='RMSE',
    eval_metric='RMSE',
    early_stopping_rounds=100,
    random_seed=42,
    verbose=100
)

# Train CatBoost model
catboost_model.fit(X_train, y_train, eval_set=(X_test, y_test))

# Make predictions with CatBoost
catboost_preds = catboost_model.predict(X_test)

# Now let's create the stacking model
estimators = [
    ('catboost', CatBoostRegressor(
        iterations=2000,
        learning_rate=0.03,
        depth=8,
        l2_leaf_reg=5,
        loss_function='RMSE',
        eval_metric='RMSE',
        early_stopping_rounds=100,
        random_seed=42,
        verbose=0
    )),
    ('xgb', XGBRegressor(random_state=42))
]

stack = StackingRegressor(
    estimators=estimators,
    final_estimator=Ridge(),
    cv=5
)

# Train the stacking model
stack.fit(X_train, y_train)

# Make predictions with stacking model
stack_preds = stack.predict(X_test)

# Evaluation function
def calculate_mape(y_true, y_pred):
    return 100 * np.mean(np.abs((y_true - y_pred) / y_true))

# Evaluate both models
print("CatBoost Results:")
print(f"MAPE: {calculate_mape(y_test, catboost_preds):.2f}%")

print("\nStacking Model Results:")
print(f"MAPE: {calculate_mape(y_test, stack_preds):.2f}%")

0:	learn: 150373.4434261	test: 120570.9058245	best: 120570.9058245 (0)	total: 4.81ms	remaining: 9.62s
100:	learn: 59195.0393056	test: 33603.6798683	best: 33603.6798683 (100)	total: 329ms	remaining: 6.18s
200:	learn: 36525.9692982	test: 25134.1523432	best: 25134.1523432 (200)	total: 655ms	remaining: 5.86s
300:	learn: 23149.7918623	test: 20958.6406048	best: 20958.6406048 (300)	total: 981ms	remaining: 5.53s
400:	learn: 14755.8261902	test: 18499.4558634	best: 18499.4558634 (400)	total: 1.3s	remaining: 5.21s
500:	learn: 9991.8166823	test: 17021.5586788	best: 17021.5586788 (500)	total: 1.63s	remaining: 4.89s
600:	learn: 7290.9796927	test: 16121.0154248	best: 16121.0154248 (600)	total: 1.96s	remaining: 4.56s
700:	learn: 5716.9819607	test: 15578.8112846	best: 15578.8112846 (700)	total: 2.27s	remaining: 4.21s
800:	learn: 4611.4615264	test: 15209.3271426	best: 15209.3271426 (800)	total: 2.59s	remaining: 3.88s
900:	learn: 3808.8601718	test: 14872.2079738	best: 14872.2079738 (900)	total: 2.91s	rem

In [611]:
print("CatBoost Results:")
print(f"MAPE: {calculate_mape(y_test, catboost_preds):.2f}%")

print("\nStacking Model Results:")
print(f"MAPE: {calculate_mape(y_test, stack_preds):.2f}%")

CatBoost Results:
MAPE: 5.43%

Stacking Model Results:
MAPE: 16.11%
